In [1]:
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from PIL import Image, ImageDraw, ImageFont
import time


model = load_model(r"C:\Users\ASUS\Desktop\best_modelrn2.keras")  




In [22]:
import cv2
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from PIL import Image

# Class labels for prediction output
class_labels = {0: "Non-Recyclable", 1: "Organic", 2: "Recyclable"}

# Open webcam
cap = cv2.VideoCapture(0)

# Initialize time tracker for predictions
last_prediction_time = 0
prediction_interval = 0.33  # seconds between predictions

# Initial prediction message
current_prediction = "Waiting for first prediction..."

# Confidence threshold
conf = 0.8  # Only show prediction if confidence is above this

# Main loop to read webcam feed
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Exit if frame not read properly

    # Get dimensions and compute center of the frame
    height, width = frame.shape[:2]
    center_x = width // 2
    center_y = height // 2
    spacing_x = 45  # Half-width of ROI box
    spacing_y = 80  # Half-height of ROI box

    # Draw a red rectangle (ROI) in the center of the frame
    cv2.line(frame, (center_x - spacing_x, center_y - spacing_y), (center_x - spacing_x, center_y + spacing_y), (0, 0, 255), 2)  
    cv2.line(frame, (center_x + spacing_x, center_y - spacing_y), (center_x + spacing_x, center_y + spacing_y), (0, 0, 255), 2) 
    cv2.line(frame, (center_x - spacing_x, center_y - spacing_y), (center_x + spacing_x, center_y - spacing_y), (0, 0, 255), 2) 
    cv2.line(frame, (center_x - spacing_x, center_y + spacing_y), (center_x + spacing_x, center_y + spacing_y), (0, 0, 255), 2) 

    # Check if it's time to make a new prediction
    current_time = time.time()
    if current_time - last_prediction_time > prediction_interval:
        # Extract ROI (Region of Interest) from the frame
        roi = frame[center_y - spacing_y:center_y + spacing_y, center_x - spacing_x:center_x + spacing_x]
        
        # Convert BGR (OpenCV format) to RGB and resize to 224x224
        roi_pil = Image.fromarray(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
        roi_pil = roi_pil.resize((224, 224))
        
        # Convert to numpy array and preprocess for ResNet
        img_array = image.img_to_array(roi_pil)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = tf.keras.applications.resnet_v2.preprocess_input(img_array)
        
        # Make prediction
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)[0]
        current_prediction = class_labels.get(predicted_class, "Unknown")
        
        # Update last prediction time
        last_prediction_time = current_time

    # Display prediction only if confidence is above threshold
    max_confidence = np.max(predictions, axis=1)[0]
    if max_confidence > conf:
        if current_prediction == "Organic":
            color = (255, 0, 0)  # Blue
        elif current_prediction == "Non-Recyclable":
            color = (0, 0, 255)  # Red
        elif current_prediction == "Recyclable":
            color = (0, 255, 0)  # Green
        else:
            color = (255, 255, 255)  # White for unknown

        # Put prediction text on frame
        cv2.putText(frame, f"Prediction: {current_prediction}", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                    1, color, 2)

    # Print confidence score to terminal for debugging
    print(max_confidence)

    # Show video feed with prediction overlay
    cv2.imshow('Waste Classification', frame)

    # Exit loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
0.64790106
0.64790106
0.64790106
0.64790106
0.64790106
0.64790106
0.64790106
0.64790106
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
0.53026384
0.53026384
0.53026384
0.53026384
0.53026384
0.53026384
0.53026384
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
0.65717113
0.65717113
0.65717113
0.65717113
0.65717113
0.65717113
0.65717113
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
0.71237105
0.71237105
0.71237105
0.71237105
0.71237105
0.71237105
0.71237105
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
0.6808178
0.6808178
0.6808178
0.6808178
0.6808178
0.6808178
0.6808178
0.6808178
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
0.5013453
0.5013453
0.5013453
0.5013453
0.5013453
0.5013453
0.5013453
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
0.64484036
0.64484036
0.64484036
0.64484036
0.64484036
0.64484036
0.64484036
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
0.6216766
0.6216766
0.6216766
0.6216766
0.6216766
0.6216766
0.6216766
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
0.66022265
0.66022265
0.66022265
